In [80]:
import pandas as pd
import geopandas
import plotly

In [73]:
us_state_to_abbrev = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC"
}

In [83]:
residents = pd.read_csv("Residency.csv", header = None)
residents.columns = ["NAME"]
residents["Residents"] = 0

for x in residents["NAME"]:
    y = x.split(" ")
    if len(y) == 3:
        state = y[0] + " " + y[1]
    else:
        state = y[0]
    number = y[-1]
    number = number[1:-1]
    residents.loc[residents["NAME"] == x, "Residents"] = number
    residents.loc[residents["NAME"] == x, "NAME"] = state

print(residents)

              NAME Residents
0         New York        72
1       California        45
2            Texas        43
3     Pennsylvania        31
4         Michigan        27
5         Illinois        23
6             Ohio        22
7          Florida        20
8    Massachusetts        18
9        Louisiana        17
10        Missouri        17
11        Maryland        16
12  North Carolina        14
13        District        13
14       Tennessee        10
15       Minnesota         9
16        Virginia         9
17         Georgia         9
18        Kentucky         8
19  South Carolina         7
20      Washington         7
21         Indiana         6
22        Colorado         6
23       Wisconsin         6
24            Iowa         5
25      New Jersey         5
26     Connecticut         5
27        Oklahoma         5
28          Oregon         5
29         Alabama         5
30            Utah         4
31     Mississippi         4
32     Puerto Rico         4
33   West Virg

In [75]:
def covid_total_pull(condition, month, years, state):
    years_all = [x for x in range(years[0], years[-1]+1)]
    months = ["0"+str(x) for x in range(1,10)] + [str(x) for x in range(10,13)]
    dates = [str(years_all[0])+"-"+months[y]+"-01" for y in range(month-1, 12)]
    if len(years_all) >=2:
        dates += [str(years_all[-1])+"-"+months[y]+"-01" for y in range(0,month)]
    if len(years_all) >2:
        for x in range(years_all[1], years_all[-1]):
            dates += [str(x)+"-"+months[y]+"-01" for y in range(0,12)]
    dates = sorted(dates)
    
    df = pd.read_csv("COVID/"+condition+"_"+state+".csv")
    try:
        df = df.drop("isPartial", axis = 1)
    except:
        return None
    df = df[df["date"].isin(dates)].mean()
    df = pd.DataFrame(df, columns = [state])
    return(df)


In [76]:
def pull_all_states(condition, month, years):
    all_data = covid_total_pull(condition, month, years, "MA")
    for x in us_state_to_abbrev:
        state = us_state_to_abbrev[x]
        if state != "Massachusetts":
            data_next = covid_total_pull(condition, month, years, state)
            if data_next is not None:
                all_data[state] = data_next[state]
            else:
                all_data[state] = 0
    return all_data

In [77]:
def all_conditions(month, years):
    kw_list = ["cataract", "diabetic retinopathy", "macular degeneration", "uveitis"]
    all_data = pull_all_states("glaucoma", month, years)
    for x in kw_list:
        all_data = pd.concat([all_data, pull_all_states(x, month, years)])
    return all_data

In [ ]:
state_to_state = all_conditions(10, [2020, 2021]).mean()
state_to_state = pd.DataFrame(state_to_state, columns = ["Interest_21"])
state_to_state["Interest_20"] = all_conditions(10, [2019, 2020]).mean()
state_to_state["Interest_19"] = all_conditions(10, [2018, 2019]).mean()
state_to_state["STUSPS"] = state_to_state.index

In [87]:
states = geopandas.read_file('data/usa-states-census-2014.shp')
states = states.drop_duplicates()
states = states.merge(state_to_state, on="STUSPS")
states = states.merge(residents, on="NAME")
print(states.columns)

Index(['STATEFP', 'STATENS', 'AFFGEOID', 'GEOID', 'STUSPS', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'region', 'geometry', 'Interest_21', 'Interest_20',
       'Interest_19', 'Residents'],
      dtype='object')


In [88]:
states["Provider_Ratio"] = states["Rate_100k_21"]/states["Residents"]

KeyError: 'Rate_100k_21'